In [1]:
import json
import os
import re
import subprocess
import pandas as pd
import numpy as np
from modules.adaptive_learner_modeling import *
from modules.learner_simulation import *
from modules.question_analysis import *
from base.llms import create_llm

gpt4o_llm = create_llm(backbone="gpt4o", deployment='gpt-4o-mini')

https://go-learning.openai.azure.com/
2024-10-21
gpt-4o-mini


In [2]:
def code_merge(codes):
    merged_code = ''
    for i,code in enumerate(codes):
        merged_code += f"The {i+1}-th submission code: {code}   \n"
    return merged_code

In [2]:
metadata_folder = 'dataset/CSEDM/SingleRecords/1/'
problems = pd.read_csv('dataset/CSEDM/problem.csv')

In [3]:
def analyze_requirement(requirement):
    return analysis_question_with_llm(gpt4o_llm, requirement)
problems['RequirementSkill'] = problems['Requirement'].apply(analyze_requirement)

# 保存回CSV文件
problems.to_csv('problem_with_skills.csv', index=False)

{'tracks': ['The problem requires basic arithmetic operations and conditional logic to handle specific cases.', 'Understanding how to define functions and return values in Java is essential for this task.'], 'result': {'Knowledge skill': {'Variables': 'beginner', 'Functions': 'beginner', 'Conditional Statements': 'beginner', 'Arithmetic Operations': 'beginner'}}}
{'tracks': ['The problem involves checking a number against certain conditions, which requires understanding of logical operators and conditionals.', 'To implement the function, knowledge of function definitions, return statements, and boolean logic is necessary.'], 'result': {'Knowledge skill': {'Variables': 'beginner', 'Conditionals': 'beginner', 'Boolean Logic': 'beginner', 'Function Definition': 'beginner', 'Return Statements': 'beginner'}}}
{'tracks': ['The problem requires understanding conditional statements to determine when to answer the phone.', 'It involves knowledge of boolean logic to return true or false based on

In [4]:
def compile_code(code, language):
    if language == 'python':
        return code
    elif language == 'java':
        start = "public class Solution {\n"
        end = "\n}"
        code = start + code + end
        with open(f'Solution.java', 'w') as f:
                f.write(code + '\n')
        com_response = subprocess.run(["javac", "Solution.java"], capture_output=True, text=True)
        if com_response.returncode != 0:
            return com_response.stderr  
        else:
            return 'There is no compilation error'
    elif language == 'cpp':
        code = code.replace('int main', 'int main')
        return code


In [ ]:
exercise_records = []
profile = ''
for i, filename in enumerate(os.listdir(metadata_folder)):
    match = re.search(r'(\d+)', filename)
    print(filename)
    result = '1'
    if match:
        result = match.group(1)
    problem_id = int(result)
    filepath = os.path.join(metadata_folder, filename)
    code_records = pd.read_csv(filepath)
    problem = problems[problems['ProblemID'] == problem_id]['Requirement'].values[0]
    merged_code = code_merge(code_records['Code'])
    last_record = f"problem stem: {problem}   \n code records: {merged_code}"
    if i<10:
        if exercise_records == []: 
            profile = initialize_learner_profile_with_llm(gpt4o_llm, merged_code)
            print(profile['learner_information'])
        else:
            profile = update_learner_profile_with_llm(gpt4o_llm, profile, merged_code)
    else:
        all_records = []
        code_record_current = code_records['Code'][0]
        all_records.append(f"1-th submission code: {code_records['Code'][0]}")
        err_info = compile_code(code_records['Code'][0], 'java')
        # code = generate_first_code_with_llm(gpt4o_llm, profile, problem, exercise_records)
        # code_records_current.append(code)
        for j in range(4):
            code = update_code_with_llm(gpt4o_llm, profile, problem, exercise_records, code_record_current, err_info)
            print(code)
            code_record_current = code_record_current.replace(code['code0'], code['code1'])
            err_info = compile_code(code_record_current, 'java')
            print(f"err_info: {err_info}")
            all_records.append(f"{j+2}-th submission code: {code_record_current}")
        with open(f'results/experiments/output_code_{problem_id}.txt', 'w') as f:
            for record in all_records:
                f.write(record + '\n')
    if len(exercise_records) < 1:
        exercise_records.append(last_record)
    else:
        exercise_records.pop(0)
        exercise_records.append(last_record)        
            

Problem_32.csv


{'tracks': [], 'result': {'learner_information': 'The learner has submitted multiple versions of the same function, showing a gradual improvement in syntax and logic, but still makes common mistakes.', 'Programming_Knowledge': {'Basic': {'Variables': 'intermediate', 'Data_Types': 'intermediate', 'Operators': 'intermediate'}, 'Control_Structures': {'If/Else': 'intermediate', 'NestedIf': 'beginner', 'Loops': 'intermediate', 'Switch_Case': 'beginner', 'Recursion': 'beginner'}, 'Data_Structures': {'Arrays': 'intermediate', 'Linked_Lists': 'beginner', 'Stacks': 'beginner', 'Queues': 'beginner', 'Trees': 'beginner'}, 'Algorithms': {'Sorting': 'beginner', 'Searching': 'beginner', 'Graph_Traversal': 'beginner'}, 'Functions': {'Function_Definition': 'intermediate', 'Function_Call': 'intermediate', 'Recursion': 'beginner'}}, 'Coding_Skills': {'Code_Writing': {'Syntax_Correctness': 'intermediate', 'Modular_Thinking': 'intermediate', 'Code_Efficiency': 'beginner'}, 'Debugging': {'Error_Identificat

In [6]:
# merged_code = code_merge(data['Code'])

In [7]:
code

{'code0': 'else if (nums[j] == 0 && j < nums.length - 1)',
 'code1': 'else if (nums[j] == 0 && j <= nums.length - 1)'}

In [8]:
import subprocess
com_response = subprocess.run(["javac", "Solution.java"])
print(com_response.stdout)
print(com_response.stderr)
response = subprocess.run(["java", "Solution"],capture_output=True, text=True)
print(response.stdout)
print(response.stderr)

None
None

Error: Main method not found in class Solution, please define the main method as:
   public static void main(String[] args)
or a JavaFX application class must extend javafx.application.Application



In [9]:
import subprocess